### Meqasa scrapper
#### This is aimed at scrapping real estate listings from meqasa using BeautifulSoup and requests

In [1]:
import re
import datetime
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [45]:
# getting just the results on the first page
def house_info(houses):
    house_names = []
    house_beds = []
    house_showers = []
    house_garages = []
    house_areas = []
    house_descriptions = []
    house_prices = []
    currencies = []
    rent_periods = []
    house_urls = []
    house_address = []
    posted_times = []
    
    for listing in houses:
        url = base_url + listing.find('a').attrs['href']
        property_name = listing.find('h2').text.strip()
        address = listing.find('h2').text.strip().split('for rent at ')[1]
        bed = isNone(listing.find('li', {'class':'bed'}))
        shower = isNone(listing.find('li',{'class': 'shower'}))  
        garage = isNone(listing.find('li', {'class': 'garage'}))
        area = isNone(listing.find('li', {'class': 'area'})) 
        price, currency, rent_period = isPriceQuoted(listing.find('p',{'class': 'h3'}).text)     
        
        
        # clicking the listing page to get the description and the updated time info
        current_item = requests.get(url)
        current_item_soup = BeautifulSoup(current_item.content, 'lxml')
        house = current_item_soup.find('div', {'class': 'bottom-one-column-details'})
        description = house.find('div', {'class': 'description'}).find('p').text
        posted_time = current_item_soup.find('p', {'class':'listed-by-text'}).text.replace('Updated on ', '').replace(' by:', '')

        
        # alright now putting everything inside append all the info
        house_names.append(property_name)
        house_beds.append(bed)
        house_showers.append(shower)
        house_garages.append(garage)
        house_areas.append(area)
        house_descriptions.append(description)
        house_prices.append(price)
        currencies.append(currency)
        rent_periods.append(rent_period)
        house_urls.append(url)
        house_address.append(address)
        posted_times.append(posted_time)
        
    # now creating a dataframe of the lists
    df_houses = pd.DataFrame({'Property' : house_names,
                              'Beds' : house_beds,
                              'Showers' : house_showers,
                              'Garages' : house_garages,
                              'Areas' : house_areas,
                              'Description' : house_descriptions,
                              'Price' : house_prices,
                              'Currency' : currencies,
                              'Rent_period' : rent_periods,
                              'Url' : house_urls,
                              'Address' : house_address,
                              'Time_posted' : posted_times})
    return(df_houses)



# function to append 0 where it doesn't have a value...
def isNone(var):
    if var is None:
        return 0
    else:
        return var.text

# creating a method to get the price, currency and rent_period from the price string based on what is searched.
def isPriceQuoted(var):
    if 'GH₵' in var:
        price = var.strip().replace('PriceGH₵', '').split('/')[0].strip()  # getting the price
        c = var.strip().replace('Price', '').split('/')[0]
        currency = re.sub('(\d+)', '',c).replace(', ', '') # getting the currency
        rent_period = var.strip().replace('PriceGH₵', '').split('/')[1].strip() # getting the rent period
    elif '$' in var:
        price = var.strip().replace('Price$', '').split('/')[0].strip()  # getting the price
        c = var.strip().replace('Price', '').split('/')[0]
        currency = re.sub('(\d+)', '',c).replace(', ', '') # getting the currency
        rent_period = var.strip().replace('Price$', '').split('/')[1].strip() # getting the rent period
    else:
        price = 'disclosed on request'
        currency = 'disclosed on request'
        rent_period = 'disclosed on request'
    return price, currency, rent_period


def save_data(df):
    file_time = datetime.datetime.now()
    file_time = file_time.strftime('%Y-%m-%d')
    fine_name = 'meqasa_'+file_time+'.csv'
    
    df.to_csv('../data/'+fine_name, index=False)

In [ ]:
df_final = pd.DataFrame([])
base_url = 'https://meqasa.com/'
results = requests.get('https://meqasa.com/houses-for-rent-in-ghana')
soup = BeautifulSoup(results.content, 'lxml')
houses = soup.find_all('div', {'class': 'row mqs-featured-prop-inner-wrap clickable'})



In [ ]:
df_final = house_info(houses)
save_data(df_final)

In [ ]:
df_final.head(10)